# SMT Solvers and the DPLL(T) Framework – Lecture Notes

## Overview

This lecture explains how SMT solvers work by integrating theory solvers directly into the **CDCL** (Conflict-Driven Clause Learning) loop used in SAT solvers. This integration improves performance over treating the SAT solver as a black box.

## Motivation

<br>

<img src="./alr-images/1120.png" width="500" style="display: block; margin: auto;">

<br>

- In earlier videos, we used the SAT solver as a black box.
- In practice, this is inefficient for large formulas with many propositional variables.

### Example Problem

Suppose:
- $x = y$ is abstracted as $b_1$
- $x < y$ is abstracted as $b_2$

During the CDCL loop, if:
- $b_1 = \text{true}$ and $b_2 = \text{true}$

This assignment might be inconsistent modulo the theory, but the SAT solver (as a black box) won’t catch it early.

## Desired Behavior

We want to **detect theory-level contradictions early**, as soon as a partial assignment becomes inconsistent.

This motivates **integrating** the theory solver into the CDCL loop.

---

## CDCL Recap (Pure SAT Solvers)

<br>

<img src="./alr-images/1121.png" width="500" style="display: block; margin: auto;">

<br>

CDCL consists of:
1. **Decide**: Choose a propositional variable and assign it true/false.
2. **Boolean Constraint Propagation (BCP)**: Apply unit propagation to find implications of assignments.
3. **Analyze Conflict**: If a conflict is found, use an implication graph to:
   - Derive a conflict clause
   - Decide a level to backtrack to

## Integrating Theory Solvers: DPLL(T)

<br>

<img src="./alr-images/1123.png" width="500" style="display: block; margin: auto;">

<br>

SMT solvers embed the theory solver directly into the CDCL process.

This is known as the **DPLL(T) framework**, where:
- **DPLL** handles SAT solving
- **T** is the theory solver module

### Workflow

<br>

<img src="./alr-images/1122.png" width="500" style="display: block; margin: auto;">

<br>

1. Perform **BCP**.
2. If BCP finds a conflict:
   - Do standard CDCL conflict analysis.
3. If BCP finds no conflict:
   - Call the **theory solver** to check consistency with the theory.
   - If inconsistent:
     - Add a **theory conflict clause** to the SAT formula.
     - Optionally use the **unsat core** of the partial assignment.

---

## Theory Propagation Lemmas

Sometimes the theory solver can detect additional **implications** that help avoid bad decisions.

### Motivation

<br>

<img src="./alr-images/1124.png" width="500" style="display: block; margin: auto;">

<br>

Example:
- SMT formula contains $x = y$, $y = z$, $x < z$
- Abstracted as $b_1$, $b_2$, $b_3$
- $b_1 = \text{true}$, $b_2 = \text{true}$

Then assigning $b_3 = \text{true}$ is clearly contradictory.

### Solution

<br>

<img src="./alr-images/1125.png" width="500" style="display: block; margin: auto;">

<br>

- The theory solver can detect that $b_1 \land b_2 \Rightarrow \lnot b_3$
- This is a **theory propagation lemma**.
- It can be added to the Boolean abstraction safely.

These lemmas:
- Are **always valid modulo the theory**
- Help prevent bad search paths

## How Are Theory Propagation Lemmas Obtained?

Two main approaches:

1. **Black-box query**: Ask theory solver whether a literal is implied by current assignment.
   - Simple but inefficient.

2. **Modified theory solver**: Implement implication detection directly.
   - More efficient.
   - Used by most competitive SMT solvers.

## Which Lemmas to Add?

Many possible lemmas could be implied. Two strategies:

1. **Exhaustive Theory Propagation**
   - Add all implied literals
   - Costly

2. **Heuristic/Selective Propagation**
   - Add only “obviously” implied literals
   - Based on heuristics
   - Most solvers use this approach

---

## Summary

- SMT solvers use the **DPLL(T)** architecture.
- The theory solver is integrated into the SAT solving loop.
- Theory conflicts and theory propagation lemmas help prune the search space.
- Efficient solvers:
  - Modify the theory solver for fast implication detection
  - Use heuristics for theory propagation
- This method supports scalable satisfiability checking in first-order theories.

---

## Conclusion

This wraps up our discussion of:
- **DPLL(T) solvers**
- **Integration of theory solvers into SAT frameworks**
- Efficient decision procedures for formulas in first-order theories

All modern SMT solvers are built on the **DPLL(T)** framework discussed here.
